## Inspect Original Data

This notebook shows how the provided data files in `data/` are derived from the original data files. Assumes that orig_data.zip (see GitHub [releases](https://github.com/chrisyeh96/voltctrl/releases/tag/v1.0)) has been unzipped into a folder called `orig_data/`.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io as spio
import scipy.stats

# hide top and right splines on plots
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

In [ ]:
# load the mat files from data folder
solar = spio.loadmat('data/PV.mat', squeeze_me=True)['actual_PV_profile']  # shape [14421]
aggr_p = spio.loadmat('data/aggr_p.mat', squeeze_me=True)['p']  # shape [14421]
aggr_q = spio.loadmat('data/aggr_q.mat', squeeze_me=True)['q']  # shape [14421]
pq_fluc = spio.loadmat('data/pq_fluc.mat', squeeze_me=True)['pq_fluc']  # shape [55, 2, 14421]
all_p = pq_fluc[:, 0]  # shape [n, T]
all_q = pq_fluc[:, 1]  # shape [n, T]

## Check load data

For whatever reason, Guanan scales the load data by a factor of 1.1.

In [ ]:
# each row in is a node (1 - 55)
# 6 columns: ['name', 'connectionkW', 'kW', 'pf', 'kVar', 'nameopal']
load = spio.loadmat('orig_data/loadavail20150908.mat', squeeze_me=True)
print(load.keys())
print(load['Load'].dtype)

scale = 1.1

In [ ]:
active_load = np.stack(load['Load']['kW']) / 1000  # to MW
active_load *= scale

In [ ]:
agg_active_load = active_load.sum(axis=0)
assert np.allclose(agg_active_load, aggr_p)

In [ ]:
reactive_load = np.stack(load['Load']['kVar']) / 1000 # to MVar
reactive_load *= scale

In [ ]:
agg_reactive_load = reactive_load.sum(axis=0)
assert np.allclose(agg_reactive_load, aggr_q)

## Check solar data

In [ ]:
solar_orig = spio.loadmat('orig_data/pvavail20150908_2.mat', squeeze_me=True)
print(solar_orig.keys())

In [ ]:
# see Load_PV_systems_3phase_delta.m
# - simulate up to 18 nodes with PV
capacities = np.array([
    9.97, 11.36, 13.53, 6.349206814, 106.142148, 154, 600, 293.54, 66.045,
    121.588489, 12.94935415, 19.35015173, 100, 31.17327501, 13.06234596,
    7.659505852, 100, 700])  # in kW
capacities /= 1000  # to MW

# for whatever reason, Guanan scales the capacities by a factor of 7
# - see line 39 in dynamic_simu_setting_revision_2nd.m
capacities *= 7

# see Generate_PV_power.m
pv_profile = solar_orig['PVavail'][0]['PVp_6s'] / solar_orig['PVavail'][0]['PVacrate']
pv = pv_profile * capacities.reshape(-1, 1)  # shape [18, 14421]

In [ ]:
# see line 41 in dynamic_simu_setting_revision_2nd.m
assert np.allclose(pv.sum(axis=0), solar)

## Check per-bus data

In [ ]:
# see dynamic_simu_setting_revision_2nd.m

# nodes with PV
# - Guanan sets substation = bus 1, then other nodes are 1,...,56
# - I use substation = bus -1, then other nodes are 0,...,54
pv_bus = np.array([9,12,14,16,19,10,11,13,15,7,2,4,20,23,25,26,32,8]) - 2
p = -active_load  # power injection
p[pv_bus] += pv

In [ ]:
assert np.allclose(p, all_p)

In [ ]:
# reactive power injection does not include any solar
assert np.allclose(-reactive_load, all_q)